# Klasterovanje

Sadržaj sveske: 

1. Šta je klasterovanje
2. K-means
3. Bisecting K-means
4. Fuzzy C-means


**Klasterovanje** se može definisati kao problem identifikacije grupa u podacima na način da su elementi u jednoj grupi(klasteru) jako slični, dok su elementi iz različitih klastera veoma različiti. Sličnost/različitost su predstavljeni nekom merom sličnosti/različitosti (npr. euklidsko rastojanje, varijansa...). Odluka koja mera sličnosti će biti korišćena zavisi od upotrebe.

Klasterovanje spada u metodu _nenadgledanog učenja_, sa obzirom na to da nemamo informacije o klasterima u kojima se nalaze instance na osnovu kojih bismo evaluirali performanse.


Pojam klasterovanja nije jednoznačno definisan. U jednom skupu se može identifikovati više različitih grupisanja.
![](Downloads/OIP.jfif)

Pojam klasterovanja nije jednoznačno definisan ne samo u odnosu na broj
klastera koji se u podacima mogu naći, već i u odnosu na to kako definišemo klastere.
Neke od vrsta su:  (Napomena: Ovde navodimo samo vrste koje će biti pokrivene na važbama)

* globularni
* gustinski
* dobro razdvojeni
* hijerarhijski
* fuzzy
* .....


## K-means

Metoda zasnovana na reprezentativnim predstavnicima. Vrši podelu podataka na disjunktne klastere.
Najpopularnije tehnike:

* K-means
* K-medoid 

K-means definiše predstavnika kao centroidu (najčeće usrednjena vrednost elemenata klastera), dok K-medoid definiše predstavnika kao medoidu (mora da bude tačka iz podataka!)

U nastavku ćemo se baviti K-means algoritmom, dok istraživanje o K-medoid ostaje za domači [literatura](https://scikit-learn-extra.readthedocs.io/en/stable/generated/sklearn_extra.cluster.KMedoids.html). 


K-means je jedna od najstarijih i najšire korišćenih algoritama za klasterovanje.

K-means predstavlja **iterativni** algoritam koji deli podatke u _K_ klastera (**broj klastera unapred definisan**).
Svaka tačka pripada **tačno jednom klasteru**.

Dodeljuje instance klasterima na način da suma kvadrata rastojanja između tačaka iz klastera i centroide bude što manja.
 Ova pretpostavka čini algoritam primenljivim samo na
podatke koji se mogu uprosečavati, poput vektora. 

Polaznih _k_ centroida se bira nasumično (mada, ako znamo nešto o
strukturi podataka, možemo ih i unapred zadati), a potom se ponavljaju
sledeći koraci:

1. rasporediti sve instance u nove klastere tako što se svaka instanca pridruži najbližoj centoridi
2. izračunati nove centroide kao prosek instanci koje su im pridružene.

Ovi koraci se izvršavaju sve dok se centroide menjaju. Kada su centroide iste
u dve uzastopne iteracije, algoritam se zaustavlja.

U okviru ovog algoritma se vrši minimizacija:

$
SSE = \sum_{i=1}^{k}\sum_{x\in C_i} d(x, c_i)^2
$
po $ c_i $, gde je **_d_** euklidsko rastojanje (ali je moguće koristiti i neko drugo).

### Pozitivne strane

 * 👍 Jednostavan, fleksibilan, efikasan. 
 * 👍 Jednostavan za interpretaciju.


### Potencijalni problemi
 * 🤔 -> 👍 _Da li postoji više jednako dobrih klastera (da li postoji više globalnih minimuma?)_ - Da, može biti više jednako dobrih podela (više globalnih minimuma).
 * 🤔 -> 💡 -> 👍   _Da li se prilikom minimizacije možemo zaglaviti u lokalnom minimumu?_ - Da, što može biti problem, ali ga rešavamo pokretanjem algoritma više puta.
 * 🤔 Pronalazi globularne klastere - ako minimizujemo eulidsko rastojanje. (može da bude problem, zavisi od problema koji rešavamo).
 * 👎 Osetljiv na autjalere - zbog kvadriranja euklidskog rastojanja.

### Primer 1 - Klasterovanje pasa

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

U uvodnom primeru ćemo koristiti skup podataka _dogs_.

In [ ]:
dogs = pd.read_csv('datasets/dogs.csv')
dogs

Pošto postoje dva numerička atributa, možemo ih i vizuelizovati. 

In [ ]:
plt.scatter(dogs['height'], dogs['weight'])
plt.xlabel('height')
plt.ylabel('weight')
plt.title("Dogs")
plt.show()

Klasterovanje ćemo izvršiti na osnovu atributa _height_ i _weight_.

In [ ]:
feature_names = dogs.columns[1:]
feature_names

In [ ]:
X = dogs[feature_names]
X

❔ Da li je potrebno izvršiti normalizaciju prilikom korišćenja K-means? (Da, zašto?)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=feature_names)
X

Nakon normalizacije vršimo klasterovanje u 2 klastera, koristeći algoritam K-means.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
kmeans = KMeans(n_clusters=2, n_init='auto')

❔ Zašto ne vršimo podelu podataka na trening i test?

In [ ]:
kmeans.fit(X)

Sada ćemo da vizuelizujemo rezultate koje smo dobili. Neke bitne informacije su nam:
   
   * Koja instanca pripada kom klasteru
   * Vrednosti centroida za svaki od klastera
   * Kolika je SSE

In [ ]:
# koja instanca pripada kom klasteru
X[kmeans.labels_ == 0]

In [ ]:

# centroide
centers = pd.DataFrame(kmeans.cluster_centers_, columns=feature_names)
centers

In [ ]:
# SSE
kmeans.inertia_

In [ ]:
plt.scatter(centers['height'], centers['weight'], marker='X', label='centroids')

for c in np.unique(kmeans.labels_):
    elems = X[kmeans.labels_ == c]
    plt.scatter(elems['height'], elems['weight'], label=c)

plt.xlabel('height')
plt.ylabel('weight')
plt.title('Two clusters with sse {}'.format(round(kmeans.inertia_, 2)))
plt.legend()
plt.show()
    

Detaljnijim pregledom klastera, vidimo da su u jednom klasteru velike rase, a u drugom srednje i male rase.

In [ ]:
dogs[kmeans.labels_ == 1]

In [ ]:
dogs[kmeans.labels_ == 0]

#### Silhouette coefficient (koeficijent senke)

**Cohesion** - meri koliko su bliski(slični) objekti iz istog kluatera (npr. SSE)

**Separation** - meri koliko su različitit objekti iz različitih klastera (npr. SE)

**Silhouette coefficient** je popularan metod evaluacije klastera koja kombinuje koheziju i separaciju.

Koeficijent senke se računa za svaku instancu pojedinačno, na sledeći način:

1. Za $i$-tu instancu računamo usrednjeno rastojanje od svih instanci iz istog klastera (Umesto rastojanja može da se koriste i druge mere različitosti). Obeležimo izračunatu vrednost sa $a_i$.

2. Za $i$-tu instancu i sve klastere koji ne sadrže $i$-tu instancu računamo usrednjeno rastojanje instance od svih elemenata iz svakog klastera. Pronalazimo minimalno rastojanje i obeležimo ga sa $b_i$.

3. silhouette coefficient za $i$-tu instancu računamo : $s_i = \frac{b_i - a_i}{max(a_i, b_i)}$

Vrednosti $s_i$ se nalazi u rasponu od -1 do 1. Negativne vrednosti nisu poželjne, pošto odgovaraju slučaju kada je $a_i > b_i$, tj. da je usrednjeno rastojanje između instanci u klasteru veće nego minimalno usrednjeno rastojanje 
od instanci iz drugih klastera.

Težimo da koeficijent senke bude pozitivan ($a_i < b_i$) i da $a_i$ bude što bliži 0.

Silhouette coefficient klastera računamo kao srednju vrednost $s_i$ instanci iz klastera.


Kao ukupna mera kvaliteta klasterovanja se dobija usrednjavanjem $s_i$ svih instanci.

In [ ]:
from sklearn.metrics import silhouette_samples

silhouette_values = silhouette_samples(X, kmeans.labels_)
silhouette_values

In [ ]:
plt.scatter(X['height'], X['weight'], c = silhouette_values, cmap="Greens")
plt.colorbar()
plt.title("Silhouette coefficient za svaku instancu za Kmeans k = 3")
plt.show()

### Izbor broja klastera

Jedan od glavnih izazova pri korišćenju Kmeans je izbor broja klastera.

In [ ]:
from sklearn.metrics import silhouette_score
ks = range(2, X.shape[0]) # [)
inits = ['random', 'k-means++']
fig = plt.figure(figsize=(10,30))
idx = 1
silhouette = []
inertias = []
for k in ks:
    for init in inits:
        kmeans = KMeans(n_clusters=k, init=init, n_init='auto')
        kmeans.fit(X)
        if init == 'k-means++':
            inertias.append(kmeans.inertia_)
            silhouette.append(silhouette_score(X, kmeans.labels_))

        fig.add_subplot(len(ks), len(inits), idx)
        idx += 1
        for label in range(k):
            cluster = dogs[kmeans.labels_ == label]
            plt.scatter(cluster['height'], cluster['weight'])
        
        centroids = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=feature_names)
        plt.scatter(centroids['height'], centroids['weight'], color='black', marker='x')
            
        plt.title(f'k={k}, init={init}, inertia={kmeans.inertia_}')
        
plt.tight_layout()

Cilj je da minimizujemo SSE(inertia). 
Sa porastom broja klastera, SSE se smanjuje, jer će instance biti bliže centroidama. SSE je najmanja kada je broj klastera jednak broju instnci (svaki klaster sadrži po 1 instancu). 
Sa obzirom na ovu informaciju, potrebno je da odredimo minimalan broj klastera takav da je razumna vrednost SSE.

#### Pravilo lakta (Elbow method) vs Silhouette coefficient

Jedan od najpoznatijih metoda je **pravilo lakta**. Prvo vizuelizujemo promenu SSE. Biramo broj klastera $k$ na "laktu", tj. u tački gde SSE najbrže opadne. Na osnovu grafika ispod, izabrali bismo k=3.

Problem koji se javlja prilikom korišćenja ove heuristike je što se klasteri koji su jako blizu spajaju u jedan (jer njihovim razdvajanjem SSE neće mnogo opasti).

Za rešenje ovog problema se koristi Silhouette coefs. Na slici desno je vizuelizovana promena silhouette coefs u odnosu na promenu broja klastera. Znamo da težimo što većim vrednostima, koje signaliziraju da je dobra koherencija unutar klastera, kao i separacija između klastera. 

Ako smo možda kod metoda lakta mogli da diskutujemo da li je bolje izabrati vrednost k=3 ili k=4, korišćenjem silhouette coefs je očigledno da je optimalno k=3.



In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(ks, inertias, marker='o')
plt.ylabel('inertia')
plt.xlabel('num_of_clusters')
plt.title('Elbow method')

plt.subplot(1, 2, 2)
plt.plot(ks, silhouette, marker="o")
plt.ylabel('silhouette coefs')
plt.xlabel('num_of_clusters')
plt.title('Silhouette coefs')

plt.show()

##### Silhouette Diagram
Još detaljnija vizuelizacija je silhouette dijagram koji se dobija vizuelizacijom silhouette coef za svaku instancu klastera, sortirane opadajuće.
Debljina svakog klastera nam daje informaciju o veličini klastera, a širina o silhouette skoru svake instance (što je šira to je bolje).

Dodatna informacija je i isprekidana linija koja predstavlja shilouette score svih klastera. 

In [ ]:
# pomoćna funkcija za silhouette diagram plot (preuzeto sa https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html)
import matplotlib.cm as cm

for n_clusters in [2, 3, 4, 5]:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.3, -0.2, -0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X['height'], X['weight'], marker="o", lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )

plt.show()

Sa leve strane se nalazi silhouette diagram, a sa desne su prikazani klasteri.
Ako posmatramo za k=2, možemo primetiti da je klaster 1 znatno veći od klastera 0.
Ako povećamo na k=3, dobijamo bolje rezultate. Ako sada posmatramo k=4, možemo primetiti da je silhouette score opao, kao i da se u klasteru 0 većina instanci nalazi sa leve strane isprekidane linije (silhouette score je manji od sr. vrednosti), čak i da je za neke instance manji od 0 (što znači da je došlo do pogrešnog razdvajanja klastera). Ovim smo još sigurniji da je k=3 najbolji izbor.

## Primer 2 - kompresija slike 

Još jedan primer upotrebe klasterovanja.
Ako želimo da kompresujemo sliku možemo smanjiti broj boja koje koristimo za prikaz.

Slika u boji predstavlja matricu piksela, koji je opisan sa 3 kanala (rgb).  Svaki piksel posmatramo kao 1 instancu i izvršimo klasterovanje u _k_ grupa. 

Nakon toga, svaki piksel zamenjujemo centroidom klastera kome pripada.

In [ ]:
from matplotlib.image import imread
import pandas as pd
import seaborn as sns

img = imread('img/masa.jpg')
img_size = img.shape


X_img = img.reshape(img_size[0] * img_size[1], img_size[2])


km = KMeans(n_init='auto', n_clusters=30)
km.fit(X_img)


X_compressed = km.cluster_centers_[km.labels_]
X_compressed = np.clip(X_compressed.astype('uint8'), 0, 255)

X_compressed = X_compressed.reshape(img_size[0], img_size[1], img_size[2])

fig, ax = plt.subplots(1, 2, figsize = (12, 8))
ax[0].imshow(img)
ax[0].set_title('Original Image')
ax[1].imshow(X_compressed)
ax[1].set_title('Compressed Image with 30 colors')
for ax in fig.axes:
    ax.axis('off')
plt.tight_layout();

## Bisecting K-means

Bisecting K-means algorithm se zasniva na ideji: Prvo podeliti instance u 2 klastera, zatim izabrati jedan od postojećih i podeliti ga na 2 klastera. Proces se ponavlja dok se ne formira $k$ klastera.

Izbor klastera za podelu se može izvršiti na više načina: možemo izabrati najveći klaster, klaster sa najvećom SSE ili kriterijum koji se zasniva na SSE i veličini. Ova odluka utiče na rezultujuće klastere.

Često se rezultujuće centroide Bisecting K-means algoritma koriste kao inicijalne centroide za klasičan K-means pronalazi algoritam. Ovaj korak je potreban pošto K-means pronalazi klastere koji predstavljaju lokalni minimum u odnosu na SSE, dok kod Bisecting K-means Kmeans koristimo samo lokalno, za podelu izabranog klastera. Dakle, finalni skup klastera nije klasterovanje koje predstavlja lokalni minimum u odnosu na SSE (na početku sveske smo definisali klasterovanje kao minimizacioni problem SSE).



In [ ]:
from sklearn. cluster import BisectingKMeans
bkmeans = BisectingKMeans(n_clusters=3,bisecting_strategy='largest_cluster')
bkmeans

###### Pomoćna funkcija za vizuelizaciju klasterovanja

In [ ]:
def visualize_clustering(data, centers, labels, feature_names, alg_name):
    plt.scatter(centers[:, 0], centers[:, 1], marker='X', label='centroids', color="black")

    for c in np.unique(labels):
        elems = X[labels == c]
        plt.scatter(elems[feature_names[0]], elems[feature_names[1]], label=c)

    plt.xlabel(feature_names[0])
    plt.ylabel(feature_names[0])
    plt.title('{} {} clusters'.format(alg_name, len(centers)))
    plt.legend()


In [ ]:
bkmeans.fit(X)

In [ ]:
visualize_clustering(X, bkmeans.cluster_centers_, bkmeans.labels_, X.columns, "Bisecting Kmeans")

## Fuzzy C - Means

Svi algoritmi sa kojima smo se do sada upoznali su **hard clustering**, tj. jedna tačka pripada najviše jednom klasteru. 
Nasuprot tome, postoje i **soft clustering** algoritmi koji dozvoljavaju da tačke pripadaju istovremeno većem broju klastera sa različitim stepenom pripadnosti (membership degree/value).

Fuzzy C-means (FCM) je jedan predstavnik soft clustering algoritama. C-means u nazivu označava $c$ centroida (identično kao kod K-means).

Kod Fuzzy C-means metoda, imamo **dva parametra** : $\mu_{ij}$ i $c_i$ i **hiper-parametare** : c i m.

* $\mu_{ij}$ - membership degree/value (stepen pripadnosti) - verovatnoća da $j$-te instanca pripada $i$-tom klasteru.
Ograničenja:
    
    * $\mu_{ij} \in [0, 1]$   $\forall i, j$
     
    
    * $\sum_{i=1}^{c} \mu_{ij} = 1$  $\forall j$
 

* $c_i$ - centroide za svaki klaster

* $m >= 1$ - fuzzifier - kontroliše koliko će granice klastera biti fuzzy.


### Minimizacioni problem

You can understand the objective function as a weighted sum of the distance between the data points (X_j) and the cluster centers (C_i). The “distance” term is the L2 norm in the equation above, and in the example

$ J = \sum_{i=1}^{C}\sum_{j=1}^{N}\mu_{ij}^md(x_j, c_i)$, gde je

$C$ - broj klastera
$N$ - broj instanci 
$x_j$ - $j$-ta instanca
$c_i$ - centroida za $i$-ti klaster
$d$ - rastojanje, npr. L2 norma (euklidsko rastojanje)

Formula predstavlja _težinsku sumu_ rastojanja između tačaka iz podataka ($x_j$) i centroide klastera ($c_i$).

Suma je "otežana" sa $\mu{ij}^m$. Pošto minimizujemo $J$, za bliže instance centroidi imamo veće vrednosti $\mu_{ij}$. Zato nam je bitan i hiper-parametar $m$.  Ako za vrednost m uzmemo jako velike vrednosti, rastojanje više nema veliki uticaj (jer je $\mu_{ij}^m$ blisko 0) i sve centroide će se nalaziti oko centra svih podataka - time se onda sve tačke nalaze u velikom broju klastera. 

Dakle, intuitivno, za veliko $m$ - instance pripadaju većem broju klastera, za malo $m$ - instance pripadaju malom broju klastera.

#### Pronalazak parametara koji minimizuju J 

[Litetatura sa izvođenjem formula.](https://www.sciencedirect.com/science/article/abs/pii/0098300484900207)

Ostalo je još pitanje pronalaska parametara $c_i$ i $\mu_{ij}$ koji minimizuju $J$:

* $c_i = \frac{\sum_{j=1}^N \mu_{ij}^mxj}{\sum_{j=1}^N \mu_{ij}^m}$
* $\mu_{ij} = \frac{1}{\sum_{k=1}^{C}(\frac{||x_j-c_i||}{||x_j-c_k||})^{\frac{2}{m-1}}}$

Dakle, Fuzzy C means iterativno ažurira $c_i$ i $\mu_{ij}$ sve dok promene centroida u dve uzastopne iteracije nije manja od unapred određene granice.

In [ ]:
#pip install scikit-fuzzy

U narednom primeru ćemo prikazati korišćenje FCM. Nakon podešavanja parametara c i m, određujemo klastere.

In [ ]:
#pip install fuzzy-c-means
from fcmeans import FCM

In [ ]:
fcm = FCM(n_clusters=3, m=3)
fcm.fit(X.to_numpy())

FCM možemo koristiti i za hard clustering, kada svaku instancu dodelimo jednom klasteru (onom za koji je stepen pripadnosti najveći).

In [ ]:
# hard clustering - dodeljivanje tacno jednom klasteru
labels = fcm.predict(X.to_numpy())

In [ ]:
centers = fcm.centers

Na osnovu vizuelizacije ispod, vidimo da smo dobili slične centroide kao sa K-means ili Bisecting K-means.

In [ ]:
visualize_clustering(X, fcm.centers, fcm.predict(X.to_numpy()), X.columns, "Fuzzy Kmeans")
plt.show()

Da bismo dobili informacije o stepenu pripadnosti različitim klasterima, koristićemo soft_predict.

In [ ]:
import seaborn as sns
sns.heatmap(fcm.soft_predict(X.to_numpy()), cmap='Greens', annot=True)
plt.title("Membership values")
plt.xlabel('clusters')
plt.ylabel('data objects')
plt.show()

 Pitanje? Da li gore pokazanim heuristikama za hard clustering možemo da biramo broj klastera za fuzzy c means?
 [Dodatna literatura](https://ieeexplore.ieee.org/document/6412415).